In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

# Fine-tuning a masked language model

There are a few cases where we want to first fine-tune the language models on our data, before training a task-specific head.

For example, if our dataset contains legal contracts or scientific articles, a vanilla Transformer model like BERT will typically treat the domain-specific words in our corpus as rare tokens, and the resulting performance may be less than satisfactory. By fine-tuning the language model on in-domain data we can boost the performance of many downstream tasks, which means we usually only have to do this step once.

This process of fine-tuning a pretrained lanauge model on in-domain data is usually called ***domain adaptation***.

## Picking a pretrained model for masked language modeling

In this section, we will use DistilBERT:

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = 'distilbert-base-uncased'
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [3]:
distilbert_num_parameters = model.num_parameters() / 1000000
print(f">>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M")
print(f">>> BERT number of parameters: 110M")

>>> DistilBERT number of parameters: 67M
>>> BERT number of parameters: 110M


In [4]:
# sample text
text = "This is a great [MASK]."

Like BERT, DistilBERT was pretrained on the English Wikipedia and BookCorpus datasets, so we expect the predictions for `[MASK]` to reflect these domains.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

With a tokenizer and a model, we can pass our text example to the model, extract the logits, and print out the top 5 candidates:

In [7]:
import torch

inputs = tokenizer(text, return_tensors='pt')
token_logits = model(**inputs).logits

# find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

# pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> This is a great deal.
>>> This is a great success.
>>> This is a great adventure.
>>> This is a great idea.
>>> This is a great feat.


## The dataset

To showcase domain adaptation, we use the Large Movie Review Dataset (or IMDb dataset).

In [ ]:
from datasets import load_dataset

imdb_dataset = load_dataset('imdb')
imdb_dataset

In [9]:
sample = imdb_dataset['train'].shuffle(seed=101).select(range(5))

for row in sample:
    print(f"\n>>> Review: {row['text']}")
    print(f">>> Label: {row['label']}")


>>> Review: What an appalling piece of rubbish!!! Who ARE all these people who blubber on about how good this is? Yes, it's "arty"; and yes, it's "foreign", but .... that's not enough. The plot is boring and disjointed, like a reality show but not so slickly made.<br /><br />The people are intrinsically uninteresting; but as characters they don't have enough depth to feel empathy for them. If they are based on real people then I feel very, very sorry for them.<br /><br />The violence (and some of it is very violent) seems quite ostentatious and gratuitous. It's like the producer has visions of being Quenton Tarantino. Not that I think very much of him, either.<br /><br />And oh yes: if I had neighbours like these, I'd move!
>>> Label: 0

>>> Review: In the Hollywood west those trail hands were a rough bunch who when they came to town, partied pretty hardy. After all trail driving was a lonely business back in the day before railroads got to all parts of the USA. <br /><br />The drover

In [10]:
# check the `unsupervised` set
unsupervised_sample = imdb_dataset['unsupervised'].shuffle(seed=101).select(range(5))

for row in unsupervised_sample:
    print(f"\n>>> Review: {row['text']}")
    print(f">>> Label: {row['label']}")


>>> Review: I liked this movie because it deals with the great questions of life. Why do we have to suffer? How much pain and suffering can a man take before he kills himself? And why does the rightous man have to suffer? And also because it shows a lot of friendship and brotherhood amongst the POW's. One of the best films I've ever seen.
>>> Label: -1

>>> Review: For a movie with a lot of silliness, this movie has soul. It's not Citizen Kane but that's not the intention. If you like Pee Wee's Big Adventure or UHF, you'll probably like this. The intent here is to entertain, nothing more...but also, nothing less. And it succeeds.<br /><br />The one word that best describes this film is "whimsical." There is, however, obvious frustration that underlies this film. You can tell that Jittlov is both a fan of watching movies as well as making them, and it's clear that he is fairly exasperated with the things that get in the way of movie-making.<br /><br />Here, he skewers the excesses of H

## Preprocesing the data

For both auto-regressive and masked language modeling, a common preprocesing step is to concatenate all the examples and then split the whole corpus into chunks of equal size.

 The reason of doing this is that individual examples might get truncated if they are too long, and that would result in losing information that might be useful for the language modeling task.

To get started, we need to tokenize our corpus *without* setting `truncation=True`.

In [11]:
def tokenize_function(examples):
    result = tokenizer(examples['text'])
    if tokenizer.is_fast:
        result['word_ids'] = [
            result.word_ids(i) for i in range(len(result['input_ids']))
        ]

    return result

In [12]:
tokenized_datasets = imdb_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text', 'label']
)

tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

Next, we group them all together and split the result into chunks. The size of the chunks depends on the amount of GPU memory. A good starting point is see what the model's maximum context size is:

In [13]:
tokenizer.model_max_length

512

This value is derived from *tokenizer_config.json* file associated with a checkpoint.

In order to run our experiments on GPUs on Colab, we will pick a smaller size:

In [14]:
chunk_size = 128

In [15]:
# slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets['train'][:3]

for idx, sample in enumerate(tokenized_samples['input_ids']):
    print(f">>> Review {idx} length: {len(sample)}")

>>> Review 0 length: 363
>>> Review 1 length: 304
>>> Review 2 length: 133


We can then concatenate all these examples with a simple dictionary comprehension:

In [16]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}

total_length = len(concatenated_examples['input_ids'])
print(f">>> Concatenated review length: {total_length}")

>>> Concatenated review length: 800


In [20]:
concatenated_examples.keys()

dict_keys(['input_ids', 'attention_mask', 'word_ids'])

In [22]:
tokenized_samples.keys()

dict_keys(['input_ids', 'attention_mask', 'word_ids'])

Now we split the concatenated reviews into chunks of the size given by `chunk_size`. To do this, we iterate over the features in `concatenated_examples` and use a list comprehension to create slices of feature feature. The result is a dictionary of chunks for each feature:

In [18]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks['input_ids']:
    print(f">>> Chunk length: {len(chunk)}")

>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 32


The last chunk will generally be smaller than the maximum chunk size. we can
* Drop the last chunk if it is smaller than `chunk_size`, or
* Pad the last chunk until its length equals `chunk_size`.

Here we apply the first approach.

In [29]:
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # compute length of contenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()

    }

    # create a new labels column
    result['labels'] = result['input_ids'].copy()

    return result

The objective in masked language modeling is to predict randomly masked tokens in the input batch, and by creating a `labels` column we provide the ground truth for our language model to learn from.

Apply `group_texts()` to our tokenized datasets:

In [31]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [32]:
tokenizer.decode(lm_datasets['train'][1]['input_ids'])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [33]:
tokenizer.decode(lm_datasets['train'][1]['labels'])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

As expected from `group_texts()`, this looks identical to the decoded `input_ids`

## Fine-tuning DistilBERT with the Trainer API

For this specific task, we need a special data collator that can randomly mask some of the tokens in each batch of texts - `DataCollatorForLanguageModeling`, with a `mlm_probability` to specify what fraction of the tokens to mask:

In [35]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

This data collator expects a list of dictionaries, where each `dict` represents a single chunk of contiguous text. We need to remove `'word_ids'` for this data collator as it does not expect it:

In [36]:
samples = [lm_datasets['train'][i] for i in range(3)]

for sample in samples:
    _ = sample.pop('word_ids')

for chunk in data_collator(samples)['input_ids']:
    print(f"\n>>> {tokenizer.decode(chunk)}")


>>> [CLS] i rented [MASK] am curious - yellow from my video store [MASK] of all the controversy that [MASK] [MASK] when it [MASK] first released in backward [MASK] i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " [MASK] really had to see this [MASK] myself. < br / > < [MASK] / > the [MASK] is centered around a young swedish drama student named lena who wants to learn everything [MASK] can [MASK] life [MASK] [MASK] particular [MASK] wants to focus her attention [MASK] to [MASK] somechon of documentary [MASK] what the average swede thought about certain political issues such

>>> as the vietnam war [MASK] race issues in the [MASK] states. in between asking politicians and [MASK] [MASK]izens [MASK] [MASK] about their opinions on [MASK], [MASK] has sex with [MASK] [MASK] teacher, [MASK], [MASK] married [MASK] [MASK] < br / > < br / > what kills [MASK] about i [MASK] curious - yello

In [37]:
for chunk in data_collator(samples)['input_ids']:
    print(f"\n>>> {tokenizer.convert_ids_to_tokens(chunk)}")


>>> ['[CLS]', 'i', 'rented', 'i', 'am', 'curious', '-', 'yellow', '[MASK]', 'my', 'video', '[MASK]', 'because', 'of', 'all', '[MASK]', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', '[MASK]', 'in', '##tr', '.', '[MASK]', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', '[MASK]', '.', 's', '.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country', ',', 'therefore', 'being', '[MASK]', 'fan', 'of', 'films', '[MASK]', '"', 'controversial', '"', 'i', 'really', 'had', 'to', '[MASK]', 'this', '[MASK]', 'myself', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'plot', 'is', 'centered', 'around', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything', 'she', 'can', 'about', 'life', '.', '[MASK]', 'particular', '[MASK]', 'wants', 'to', 'focus', 'her', 'attention', '##s', 'to', 'gibbs', 'some', '[MASK]', 'of', 'documentary', 'on', '[MASK]', 'the', 'average', 'sw', '##ede', '

When training models for masked language modeling, one technique is to mask whole words together, not just individual tokens. This is called *whole word masking*.

To achieve this, we use the word IDs computed earlier to make a map between word indices and the corresponding tokens, then randomly decide which words to mask and apply that mask on thei nputs. Note that the labels are all `-100` except for the ones corresponding to mask wordss:

In [38]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2 # whole word masking probability

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop('word_ids')

        # create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_idx = -1
        current_word = None

        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_idx += 1

                mapping[current_word_idx].append(idx)

        # randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature['input_ids']
        labels = feature['labels']
        new_labels = [-100] * len(labels)

        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

        feature['labels'] = new_labels

    return default_data_collator(features)

In [39]:
samples = [lm_datasets['train'][i] for i in range(3)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch['input_ids']:
    print(f"\n>>> {tokenizer.decode(chunk)}")


>>> [CLS] i rented [MASK] am curious - yellow from my video store because of [MASK] the controversy [MASK] surrounded it when it was first released in 1967. i also heard that at [MASK] it was [MASK] by u. s. customs if it ever [MASK] to enter this country, therefore being [MASK] fan [MASK] films [MASK] " controversial " i really had to see this for myself [MASK] < [MASK] / > < br / > the plot is centered around [MASK] young swedish drama [MASK] named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on [MASK] the average swede thought [MASK] certain political issues such

>>> as the vietnam war and race issues in the united states [MASK] in [MASK] asking politicians and ordinary denizens of stockholm [MASK] [MASK] [MASK] on politics, she has sex with [MASK] drama [MASK], classmates [MASK] and married men [MASK] < br / [MASK] < br / [MASK] what [MASK] [MASK] about i am curious - [MASK] is that 40 ye

In [40]:
train_size = 10000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets['train'].train_test_split(
    train_size=train_size, test_size=test_size, seed=101
)

downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [41]:
from transformers import TrainingArguments

batch_size = 64
logging_steps = len(downsampled_dataset['train']) // batch_size
model_name = model_checkpoint.split('/')[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=True,
    logging_steps=logging_steps,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We use `logging_steps` to track the training loss with each epoch. We also use `fp16=True` to enable mixed-precision training.

The `Trainer` will by default remove any columns that are not part of the model's `forward` method. If we are using the whole word masking collator, we need to set `remove_unused_columns=False` to ensure we do not lose the `word_ids` column during training.

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset['train'],
    eval_dataset=downsampled_dataset['test'],
    data_collator=data_collator, # use standard data collator,
    tokenizer=tokenizer,
)

### Perplexity for language models

Assuming our test set consists mostly of sentences that are grammatically correct, then one way to measure the quality of our language model is to calculate the probabilities it assigns to the next word in all the sentences of the test set. High probabilities indicates that the model is not “surprised” or “perplexed” by the unseen examples, and suggests it has learned the basic patterns of grammar in the language. There are various mathematical definitions of perplexity, but the one we’ll use defines it as the exponential of the cross-entropy loss.

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.3f}")

A lower perplexity score means a better language model.

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.3f}")

## Fine-tuning DistilBERT with Accelerate

The `DataCollatorForLanguageModeling` applies random masking with each evaluation, so we will see some fluctuations in our perplexity scores with each training run.

One way to eliminate this source of randomness is to apply the masking once on the whole test set, and then use the default data collator in Transformers to collect the batches during evaluation.

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)

    # create a new "masked" column for each column in the dataset
    new_batch = {"masked_" + k: v.numpy() for k,v in masked_inputs.items()}

    return new_batch

Then, we apply this function to our test set and drop the unmasked columns so we can replace them with the masked ones.

In [ ]:
downsampled_dataset = downsampled_dataset.remove_columns(['word_ids'])
eval_dataset = downsampled_dataset['test'].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset['test'].column_names,
)

eval_dataset = eval_dataset.rename_columns(
    {
        'masked_input_ids': 'input_ids',
        'masked_attention_mask': 'attention_mask',
        'masked_labels': 'labels',
    }
)

Then we can set up the dataloader as usual

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=batch_size,
    collate_fn=default_data_collator,
)

In [ ]:
# create a model instance
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
from torch.optim import AdamW

# optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Prepare Accelerator
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
model_name = 'distilbert-base-uncased-finetuned-imdb-accelerate'

In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float('inf')

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # save
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

## Using our fine-tuned model

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="huggingface-course/distilbert-base-uncased-finetuned-imdb"
)

In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")